In [7]:
import pandas as pd
from pathlib import Path

src = "data/raw/census/98-401-X2021006_English_CSV_data_Ontario.csv"
out_wide = "data/processed/census_profile_on_DA_wide.csv"


In [8]:
df_head = pd.read_csv(src, nrows=5, encoding='latin1')
print(df_head.columns)   # show first 40 columns

Index(['CENSUS_YEAR', 'DGUID', 'ALT_GEO_CODE', 'GEO_LEVEL', 'GEO_NAME',
       'TNR_SF', 'TNR_LF', 'DATA_QUALITY_FLAG', 'CHARACTERISTIC_ID',
       'CHARACTERISTIC_NAME', 'CHARACTERISTIC_NOTE', 'C1_COUNT_TOTAL',
       'SYMBOL', 'C2_COUNT_MEN+', 'SYMBOL.1', 'C3_COUNT_WOMEN+', 'SYMBOL.2',
       'C10_RATE_TOTAL', 'SYMBOL.3', 'C11_RATE_MEN+', 'SYMBOL.4',
       'C12_RATE_WOMEN+', 'SYMBOL.5'],
      dtype='object')


In [9]:
# Set the option to display all columns
pd.set_option('display.max_columns', None)

# You can also set the maximum column width to prevent truncation of long values
pd.set_option('display.max_colwidth', None)
df_head.head(2)

,CENSUS_YEAR,DGUID,ALT_GEO_CODE,GEO_LEVEL,GEO_NAME,TNR_SF,TNR_LF,DATA_QUALITY_FLAG,CHARACTERISTIC_ID,CHARACTERISTIC_NAME,CHARACTERISTIC_NOTE,C1_COUNT_TOTAL,SYMBOL,C2_COUNT_MEN+,SYMBOL.1,C3_COUNT_WOMEN+,SYMBOL.2,C10_RATE_TOTAL,SYMBOL.3,C11_RATE_MEN+,SYMBOL.4,C12_RATE_WOMEN+,SYMBOL.5
0,2021,2021A000011124,1,Country,Canada,3.1,4.3,20000,1,"Population, 2021",1.0,36991981.0,NaN,NaN,...,NaN,...,NaN,...,NaN,...,NaN,...
1,2021,2021A000011124,1,Country,Canada,3.1,4.3,20000,2,"Population, 2016",1.0,35151728.0,NaN,NaN,...,NaN,...,NaN,...,NaN,...,NaN,...


In [ ]:
import pandas as pd
src = "data/raw/census/98-401-X2021006_English_CSV_data_Ontario.csv"

peek = pd.read_csv(src, nrows=50_000, encoding="latin1",
                   usecols=["GEO_LEVEL","ALT_GEO_CODE","CHARACTERISTIC_ID",
                            "CHARACTERISTIC_NAME","C1_COUNT_TOTAL","C10_RATE_TOTAL"])
print("distinct GEO_LEVELs:", sorted(peek["GEO_LEVEL"].dropna().unique().tolist())[:10])

# See what 'Population' / 'household' / 'median' strings look like in file
for kw in ["Population", "household", "Median total income of households"]:
    print(f"\n--- candidates for: {kw} ---")
    print(peek.loc[peek["CHARACTERISTIC_NAME"].str.contains(kw, case=False, na=False),
                   "CHARACTERISTIC_NAME"].drop_duplicates().head(20).to_string(index=False))


distinct GEO_LEVELs: ['Census division', 'Census subdivision', 'Country', 'Dissemination area', 'Province']

--- candidates for: Population ---
                                                                                                              Population, 2021
                                                                                                              Population, 2016
                                                                                    Population percentage change, 2016 to 2021
                                                                                       Population density per square kilometre
                                                                              Total - Age groups of the population - 100% data
                                                    Total - Distribution (%) of the population by broad age groups - 100% data
                                                                                              

In [14]:
import pandas as pd
from pathlib import Path

# INPUT / OUTPUT
SRC = "data/raw/census/98-401-X2021006_English_CSV_data_Ontario.csv"
LONG_OUT = "data/processed/census_on_DA_long.csv"
WIDE_OUT = "data/processed/census_on_DA_wide.csv"

Path("data/processed").mkdir(parents=True, exist_ok=True)

# --- Helpers to normalize strings ---
def norm(s: pd.Series) -> pd.Series:
    # strip, collapse multiple spaces to one, lowercase
    return (s.astype(str)
              .str.strip()
              .str.replace(r"\s+", " ", regex=True)
              .str.lower())

# Target characteristic names (normalized)
# Note: you said your file has leading spaces and "household" (singular).
TARGETS_CANON_TO_OUT = {
    "population, 2021": "Population_2021",
    "total - private households by household size - 100% data": "Total_households",
    "median total income of household in 2020 ($)": "Median_income_2020",
}

# Columns to read (keeps memory down)
USECOLS = [
    "GEO_LEVEL",
    "ALT_GEO_CODE",           # DAUID for DA rows
    "CHARACTERISTIC_NAME",
    "C1_COUNT_TOTAL",         # counts (population/households)
    "C10_RATE_TOTAL"          # medians/rates (median income)
]

# Initialize the long output file with header
pd.DataFrame(columns=["DAUID", "characteristic", "value"]).to_csv(LONG_OUT, index=False)

chunksize = 500_000
for i, chunk in enumerate(pd.read_csv(
        SRC,
        encoding="latin1",
        usecols=USECOLS,
        chunksize=chunksize,
        low_memory=False)):

    # Normalize GEO_LEVEL and filter DA rows
    geo_level = norm(chunk["GEO_LEVEL"])
    chunk = chunk[geo_level.eq("dissemination area")]

    if chunk.empty:
        print(f"chunk {i+1}: no DA rows, skipped")
        continue

    # Normalize characteristic names for matching
    char_norm = norm(chunk["CHARACTERISTIC_NAME"])

    # Keep only target characteristics (normalized match)
    keep_mask = char_norm.isin(TARGETS_CANON_TO_OUT.keys())
    chunk = chunk[keep_mask].copy()
    if chunk.empty:
        print(f"chunk {i+1}: no target characteristics, skipped")
        continue

    # Coalesce value: prefer counts, else rate/median
    chunk["value"] = chunk["C1_COUNT_TOTAL"].where(
        chunk["C1_COUNT_TOTAL"].notna(),
        chunk["C10_RATE_TOTAL"]
    )

    # Build normalized characteristic label (canonical)
    chunk["characteristic"] = char_norm.map(TARGETS_CANON_TO_OUT)

    # DAUID as string with leading zeros
    chunk["DAUID"] = chunk["ALT_GEO_CODE"].astype(str).str.zfill(8)

    # Keep only the essentials
    out_long = chunk.loc[:, ["DAUID", "characteristic", "value"]]

    # Append to disk (so we never hold everything in RAM)
    out_long.to_csv(LONG_OUT, mode="a", header=False, index=False)

    print(f"✅ chunk {i+1} appended: {len(out_long)} rows")

# --- Pivot long → wide (this file is now small enough for RAM) ---
long_df = pd.read_csv(LONG_OUT)
wide = long_df.pivot_table(
    index="DAUID",
    columns="characteristic",
    values="value",
    aggfunc="first"
).reset_index()

# Ensure numeric dtype
for col in ["Population_2021", "Total_households", "Median_income_2020"]:
    if col in wide.columns:
        wide[col] = pd.to_numeric(wide[col], errors="coerce")

# Save final wide table
wide.to_csv(WIDE_OUT, index=False)
print("✅ Saved:", WIDE_OUT, wide.shape)


✅ chunk 1 appended: 548 rows
✅ chunk 2 appended: 535 rows
✅ chunk 3 appended: 564 rows
✅ chunk 4 appended: 570 rows
✅ chunk 5 appended: 570 rows
✅ chunk 6 appended: 570 rows
✅ chunk 7 appended: 570 rows
✅ chunk 8 appended: 570 rows
✅ chunk 9 appended: 570 rows
✅ chunk 10 appended: 552 rows
✅ chunk 11 appended: 522 rows
✅ chunk 12 appended: 555 rows
✅ chunk 13 appended: 534 rows
✅ chunk 14 appended: 555 rows
✅ chunk 15 appended: 519 rows
✅ chunk 16 appended: 543 rows
✅ chunk 17 appended: 546 rows
✅ chunk 18 appended: 561 rows
✅ chunk 19 appended: 567 rows
✅ chunk 20 appended: 567 rows
✅ chunk 21 appended: 570 rows
✅ chunk 22 appended: 567 rows
✅ chunk 23 appended: 552 rows
✅ chunk 24 appended: 571 rows
✅ chunk 25 appended: 568 rows
✅ chunk 26 appended: 567 rows
✅ chunk 27 appended: 568 rows
✅ chunk 28 appended: 564 rows
✅ chunk 29 appended: 552 rows
✅ chunk 30 appended: 570 rows
✅ chunk 31 appended: 570 rows
✅ chunk 32 appended: 570 rows
✅ chunk 33 appended: 570 rows
✅ chunk 34 appended

In [1]:
import pandas as pd
import geopandas as gpd
from pathlib import Path

# Paths
DA_SHAPE = "data/raw/geo/lda_000a21a_e/lda_000a21a_e.shp"   # 2021 DA Digital Boundary
DEMO_CSV = "data/processed/census_on_DA_wide.csv"
ISO_SHAPE = "data/raw/geo/branches_isochrones.shp"

OUT_TABLE = "data/outputs/trade_area_demographics.csv"
OUT_GEO   = "data/outputs/trade_area_demographics.geojson"
OUT_SHP   = "data/outputs/trade_area_demographics.shp"
Path("data/outputs").mkdir(parents=True, exist_ok=True)

# Load DAs and demo
da = gpd.read_file(DA_SHAPE)
demo = pd.read_csv(DEMO_CSV, dtype={"DAUID":"string"})

# Normalize keys
da["DAUID"]  = da["DAUID"].astype(str).str.zfill(8)
demo["DAUID"] = demo["DAUID"].str.zfill(8)

# (Robust Ontario filter)
#  - Prefer PRNAME if present
#  - else use DAUID prefix (Ontario = "35")
if "PRNAME" in da.columns:
    da_on = da[da["PRNAME"].eq("Ontario")].copy()
else:
    da_on = da[da["DAUID"].str.startswith("35")].copy()

# Join attributes
gda = da_on.merge(demo, on="DAUID", how="inner")
print("DAs joined:", len(gda))


DAs joined: 20408


2) Prep CRS for correct areas (projected meters)

Keep computations in EPSG:3347 (NAD83 / StatsCan Lambert) for accurate areas.

In [2]:
# Ensure projected CRS (3347) for area math
if gda.crs is None or gda.crs.to_epsg() not in (3347,):
    gda = gda.to_crs(3347)

# Original DA areas (used for area-weighting)
gda["DA_area_m2"] = gda.geometry.area


3) Read your isochrones and match CRS

In [3]:
iso = gpd.read_file(ISO_SHAPE)
if iso.crs is None or iso.crs.to_epsg() != 3347:
    iso = iso.to_crs(3347)

# (Optional) dissolve tiny slivers & simplify grouping keys
# Keep only what we need for grouping
keep_cols = [c for c in iso.columns if c in ("FacilityID","ToBreak","FromBreak","Name")]
iso = iso[keep_cols + ["geometry"]].copy()

# Dissolve per FacilityID & ToBreak to reduce overlay complexity (optional but helpful)
iso_diss = iso.dissolve(by=["FacilityID","ToBreak"], as_index=False)


In [ ]:
# Spatial overlay
inter = gpd.overlay(gda, iso_diss, how="intersection")

# Areas & weights
inter["part_area_m2"] = inter.geometry.area
inter["w"] = (inter["part_area_m2"] / inter["DA_area_m2"]).clip(upper=1).fillna(0)

# Ensure numeric
for col in ("Population_2021","Total_households","Median_income_2020"):
    if col in inter.columns:
        inter[col] = pd.to_numeric(inter[col], errors="coerce")

# Weighted aggregation per branch/time band
def _agg(df):
    w = df["w"]
    out = {}
    if "Population_2021" in df:
        out["Population_2021"] = (df["Population_2021"] * w).sum()
    if "Total_households" in df:
        out["Total_households"] = (df["Total_households"] * w).sum()
    if "Median_income_2020" in df:
        # area-weighted mean (proxy). If you want HH-weighted, use Total_households * w instead of w.
        out["Median_income_2020_wavg"] = (df["Median_income_2020"] * w).sum() / (w.sum() or 1)
    out["DAs_covered"] = df["DAUID"].nunique()
    out["Area_m2"] = df["part_area_m2"].sum()
    return pd.Series(out)

agg = (inter.groupby(["FacilityID","ToBreak"], as_index=False)
             .apply(_agg)
             .reset_index(drop=True))

# Nice ordering of bands (if ToBreak is numeric-like)
try:
    agg["ToBreak"] = agg["ToBreak"].astype(float).astype(int)
    agg = agg.sort_values(["FacilityID","ToBreak"])
except Exception:
    pass

# Save table
agg.to_csv(OUT_TABLE, index=False)
print("Saved table:", OUT_TABLE)

# (Optional) save a geo layer of intersected pieces for QA maps
inter.to_file(OUT_GEO, driver="GeoJSON")
inter.to_file(OUT_SHP, driver="ESRI Shapefile")
print("✅ Saved GeoJSON and SHP in data/outputs/")


/tmp/ipykernel_315127/3564444109.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_agg)
/tmp/ipykernel_315127/3564444109.py:45: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  inter.to_file(OUT_SHP, driver="ESRI Shapefile")
/home/dylannguyen/anaconda3/envs/geopanda/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Median_income_2020' to 'Median_inc'
  ogr_write(
/home/dylannguyen/anaconda3/envs/geopanda/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Population_2021' to 'Population'
  ogr_write(
/home/dylannguyen/anaconda3/envs/geopanda/li

Saved table: data/outputs/trade_area_demographics.csv
✅ Saved GeoJSON and SHP in data/outputs/


In [ ]:
inter.to_file("data/outputs/trade_area_demographics.shp", driver="ESRI Shapefile")
print("✅ Saved GeoJSON and SHP in data/outputs/")

In [7]:
print(agg.head())
print(agg.groupby("ToBreak")[["Population_2021","Total_households","Area_m2"]].sum())

   FacilityID  ToBreak  Population_2021  Total_households  \
0           1        5     49366.651118      25589.675273   
1           1       10    101681.061862      43135.901684   
2           1       15    216451.162989      98861.121962   
3           2        5     48781.379375      17364.216567   
4           2       10    261300.419289      91768.718319   

   Median_income_2020_wavg  DAs_covered       Area_m2  
0            136414.097184         74.0  4.981015e+06  
1            169014.367547        218.0  2.018371e+07  
2            129920.023271        438.0  4.827144e+07  
3             87636.386778         96.0  1.254643e+07  
4             90529.539601        503.0  7.169699e+07  
         Population_2021  Total_households       Area_m2
ToBreak                                                 
5           4.611406e+05      1.913374e+05  1.318974e+08
10          1.776814e+06      6.696874e+05  7.250504e+08
15          2.804371e+06      1.050634e+06  1.835346e+09


In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path


In [2]:
# paths (adjust to your folders)
DEMAND = "data/processed/Demand_Centers.shp"         # output from Mean Center (1 point per cluster)
BRANCH  = "data/branches/branches.shp"               # your branch points
DEMOCSV = "data/outputs/trade_area_demographics.csv" # optional (for attractiveness from 5-min band)

dc = gpd.read_file(DEMAND)
br = gpd.read_file(BRANCH)
dc = dc.rename(columns={"SUM_Popula":"pop_w"})  # ensure population weight column name
if "pop_w" not in dc.columns:
    raise ValueError("Demand centers must include SUM_Population_2021 (renamed to pop_w).")

# Keep only needed cols
dc = dc[["CLUSTER_ID","pop_w","geometry"]].copy()
if "BranchID" in br.columns: 
    br["FacilityID"] = br.index.astype(str)
elif "BranchID" not in br.columns:
    br = br.rename(columns={"FacilityID":"BranchID"})

br["StoreAttr"] = (
    br["StoreAttr"]
    .astype(str)            # ensure string
    .str.replace(",", "", regex=False)  # remove commas
    .astype(float)          # convert to float (or int if safe)
)
br["StoreAttr"] = pd.to_numeric(br["StoreAttr"], errors="raise")
br = br[["BranchID","geometry", "StoreAttr"]].copy()

# Ensure both layers are in WGS84 for haversine
if dc.crs is None or dc.crs.to_epsg() != 4326: dc = dc.to_crs(4326)
if br.crs is None or br.crs.to_epsg() != 4326: br = br.to_crs(4326)


In [3]:
br.head()

,BranchID,geometry,StoreAttr
0,1,POINT (-79.3936 43.7128),239199.0
1,2,POINT (-79.2331 43.7813),279304.0
2,3,POINT (-79.5452 43.6382),197907.0
3,4,POINT (-79.6441 43.589),200416.0
4,5,POINT (-79.7632 43.7315),223465.0


In [3]:
# vectorized haversine
def hav_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return 2*R*np.arcsin(np.sqrt(a))

dc_coords = np.vstack([dc.geometry.y.values, dc.geometry.x.values]).T  # lat, lon
br_coords = np.vstack([br.geometry.y.values, br.geometry.x.values]).T

# pairwise distances: (n_demand x n_branches)
dist_mat = np.empty((dc_coords.shape[0], br_coords.shape[0]), dtype="float64")
for j, (lat2, lon2) in enumerate(br_coords):
    dist_mat[:, j] = hav_km(dc_coords[:,0], dc_coords[:,1], lat2, lon2)


In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path

beta = 1.8  # distance decay

A = br["StoreAttr"].values.reshape(1, -1)          # (1 x nb)
D = np.clip(dist_mat, 0.1, None)                    # avoid divide by zero (0.1 km minimum)

W = A / (D ** beta)                                 # (nd x nb)
den = W.sum(axis=1, keepdims=True)                  # sum over branches per demand
P = W / np.where(den==0, 1, den)                    # probabilities P_ij

# Expected customers from each demand center to each branch
pop_w = dc["pop_w"].values.reshape(-1, 1)           # (nd x 1)
E = P * pop_w                                       # expected_cust_ij

# Long table for export
ij = np.transpose(np.nonzero(np.ones_like(P, dtype=bool)))  # all pairs
out = pd.DataFrame({
    "OriginID": dc["CLUSTER_ID"].values[ij[:,0]],
    "BranchID": br["BranchID"].values[ij[:,1]],
    "prob": P[ij[:,0], ij[:,1]],
    "expected_cust": E[ij[:,0], ij[:,1]],
    "distance_km": D[ij[:,0], ij[:,1]],
    "StoreAttr": br["StoreAttr"].values[ij[:,1]]
})

# Branch market share
market = (out.groupby("BranchID", as_index=False)
             .agg(expected_cust=("expected_cust","sum")))
market["market_share_%"] = 100 * market["expected_cust"] / market["expected_cust"].sum()

# Dominant branch per demand center (for mapping)
max_idx = np.argmax(P, axis=1)
dc_dom = dc[["CLUSTER_ID","geometry"]].copy()
dc_dom["DomBranch"] = br["BranchID"].values[max_idx]
dc_dom["DomProb"]   = P[np.arange(P.shape[0]), max_idx]
dc_dom["PopWeighted"] = pop_w.ravel()


In [5]:
Path("data/outputs").mkdir(parents=True, exist_ok=True)

out.to_csv("data/outputs/huff_od_probabilities.csv", index=False)
market.to_csv("data/outputs/huff_market_share.csv", index=False)
dc_dom.to_file("data/outputs/huff_demand_dominant.geojson", driver="GeoJSON")  # quick map QA


In [9]:
gdf = dc_dom.copy()
gdf["lat"] = gdf.geometry.y
gdf["lon"] = gdf.geometry.x
gdf.drop(columns="geometry").to_csv("data/outputs/huff_demand_dominant.csv", index=False)